In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np


class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 20) 
        self.layer2 = nn.Linear(20, 1)

    def forward(self, x): 
        return self.layer2(self.layer1(x))


model = LinearModel()

In [29]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

loss_fn = torch.nn.MSELoss()

In [30]:
# Define a custom dataset
class RandomNormalDataset(Dataset):
    def __init__(self, size=1000, mean=0, std=1):
        self.data = torch.tensor(np.random.normal(loc=mean, scale=std, size=(size, 2)), dtype=torch.float32)
        self.labels = self.data[:, 0] + self.data[:, 1]  # y = x1 + x2
        self.size = size

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Create an instance of the dataset
random_dataset = RandomNormalDataset(size=1000, mean=0, std=1)

# Create a DataLoader
batch_size = 32
dataloader = DataLoader(random_dataset, batch_size=batch_size, shuffle=True)

# Iterate over the DataLoader
for inputs, labels in dataloader:
    # Your training/validation/testing logic here
    print("Inputs shape:", inputs.shape)
    print("Labels shape:", labels.shape)

Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size([32, 2])
Labels shape: torch.Size([32])
Inputs shape: torch.Size(

In [45]:
for inputs, labels in dataloader:
    # Your training/validation/testing logic here
    optimizer.zero_grad()
    output = model(inputs)
    loss = loss_fn(output, labels)
    loss.backward()
    
    optimizer.step()
    
    
    
    

In [51]:
for group in optimizer.param_groups:
    #print(1, group)
    params_with_grad = []
    grads = []
    state_steps = []
    #beta1, beta2 = group['betas']
    for p in group['params']:
        if p.grad is None:
            continue
        params_with_grad.append(p)
        #print(1, p)
        state = optimizer.state[p]
        if len(state) == 0:
            state['step'] = torch.zeros((1,), dtype=torch.float, device=p.device) \
                if optimizer.defaults['capturable'] else torch.tensor(0.)
            state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
            state['hessian'] = torch.zeros_like(p, memory_format=torch.preserve_format)
        print(optimizer.state)
        grads.append(p.grad)


defaultdict(<class 'dict'>, {Parameter containing:
tensor([[ 0.5382, -0.1223],
        [ 0.1597,  0.5839],
        [-0.6093,  0.1333],
        [-0.5638,  0.0208],
        [-0.6340, -0.1491],
        [ 0.2697,  0.0894],
        [ 0.1178, -0.1750],
        [-0.5787,  0.0585],
        [-0.1004,  0.6915],
        [ 0.0168, -0.2897],
        [ 0.0191, -0.0695],
        [-0.4747,  0.6366],
        [ 0.3025, -0.6353],
        [-0.6474,  0.4445],
        [ 0.0980, -0.3781],
        [ 0.2990, -0.1777],
        [ 0.1695,  0.5022],
        [-0.1397,  0.6627],
        [ 0.4554, -0.6425],
        [ 0.2367,  0.5436]], requires_grad=True): {'momentum_buffer': None}, Parameter containing:
tensor([ 0.5078,  0.5229, -0.0511, -0.4995, -0.1596,  0.5541,  0.3591,  0.0290,
        -0.1168,  0.6964,  0.6512, -0.0388, -0.5495,  0.1115, -0.3719,  0.1351,
        -0.0768,  0.1364,  0.6966, -0.1278], requires_grad=True): {'momentum_buffer': None}, Parameter containing:
tensor([[-0.2122,  0.2109, -0.0678,  0.1445

In [57]:
print(params_with_grad[0].sign(), grads[0])

tensor([[ 1., -1.],
        [ 1.,  1.],
        [-1.,  1.],
        [-1.,  1.],
        [-1., -1.],
        [ 1.,  1.],
        [ 1., -1.],
        [-1.,  1.],
        [-1.,  1.],
        [ 1., -1.],
        [ 1., -1.],
        [-1.,  1.],
        [ 1., -1.],
        [-1.,  1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1.,  1.],
        [-1.,  1.],
        [ 1., -1.],
        [ 1.,  1.]], grad_fn=<SignBackward0>) tensor([[ 0.2656,  0.0238],
        [-0.2609, -0.0234],
        [ 0.0833,  0.0075],
        [-0.1814, -0.0162],
        [-0.0975, -0.0087],
        [-0.2157, -0.0193],
        [ 0.0960,  0.0086],
        [ 0.0919,  0.0082],
        [ 0.2178,  0.0195],
        [-0.0290, -0.0026],
        [ 0.1457,  0.0130],
        [-0.0713, -0.0064],
        [ 0.2321,  0.0208],
        [-0.0945, -0.0085],
        [ 0.2612,  0.0234],
        [-0.1110, -0.0099],
        [ 0.2000,  0.0179],
        [ 0.2756,  0.0247],
        [-0.1207, -0.0108],
        [-0.0682, -0.0061]])
